In [10]:
import open3d as o3d
import numpy as np

def pca_compute(data, sort=True):
    """
     SVD分解计算点云的特征值与特征向量
    :param data: 输入数据
    :param sort: 是否将特征值特征向量进行排序
    :return: 特征值与特征向量
    """
    average_data = np.mean(data, axis=0)  # 求均值
    decentration_matrix = data - average_data  # 去中心化
    H = np.dot(decentration_matrix.T, decentration_matrix)  # 求解协方差矩阵 H
    eigenvectors, eigenvalues, eigenvectors_T = np.linalg.svd(H)  # SVD求解特征值、特征向量

    if sort:
        sort = eigenvalues.argsort()[::-1]  # 降序排列
        eigenvalues = eigenvalues[sort]  # 索引
    return eigenvalues

def caculate_surface_curvature(cloud, k_neighbors=10):
    """
    计算点云的表面曲率
    :param cloud: 输入点云
    :param k_neighbors: 最近邻的数量
    :return: 点云中每个点的表面曲率
    """
    points = np.asarray(cloud.points)
    kdtree = o3d.geometry.KDTreeFlann(cloud)
    num_points = len(cloud.points)
    curvature = []  # 储存表面曲率
    for i in range(num_points):
        k, idx, _ = kdtree.search_knn_vector_3d(cloud.points[i], k_neighbors)  # 使用search_knn_vector_3d函数
        neighbors = points[idx, :]
        w = pca_compute(neighbors)  # w为特征值
        delt = np.divide(w[2], np.sum(w), out=np.zeros_like(w[2]), where=np.sum(w) != 0)
        curvature.append(delt)
    curvature = np.array(curvature, dtype=np.float64)
    return curvature

# 创建一个包含几个点的示例坐标数组
points = np.random.rand(2048, 3)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
k_neighbors = 5  # 设置最近邻的数量
surface_curvature = caculate_surface_curvature(pcd, k_neighbors)
print(surface_curvature)  # 输出表面曲率

[0.01966448 0.03807622 0.06450703 ... 0.03149882 0.02098065 0.04488783]
